In [71]:
from pymongo import MongoClient
import pandas as pd
import re

In [72]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [73]:
db, coll = connectCollection('companies','companies')

In [74]:
query = coll.find({"$and": [
                                        {"offices": {"$exists": True}},
                                        {"offices": {"$ne": None}},
                                        {"deadpooled_year": {"$type": 10}},
                                        {"number_of_employees": {"$gte": 20}},
                                        {"founded_year":{"$gte": 2007}},
                                        {"$or":[{"offices.latitude":{"$ne":"null"}},
                                        {"offices.longitude":{"$ne":"null"}}]},
                                        {"$or": [
                                        {"category_code": "games_video"},
                                        {"category_code": "software"},
                                        {"category_code": "web"},
                                        {"category_code": "mobile"},
                                        {"category_code": "social"},
                                        {"category_code": "photo_video"},
                                        {"category_code": "analytics"},
                                        ]},
                                    ]
                                    },
                                    {"_id": 0, "crunchbase_url": 0, "permalink": 0, "homepage_url": 0,
                                     "blog_url": 0, "blog_feed_url": 0, "twitter_username": 0, "founded_month":0,
                                     "founded_day": 0, "deadpooled_month":0, "deadpooled_day": 0, "deadpooled_url":0,
                                     "ipo":0, "tag_list":0, "alias_list":0, "email_address":0, "phone_number":0,
                                     "created_at":0, "updated_at":0, "overview":0, "relationships":0, "competitions":0,
                                     "providerships":0, "funding_rounds":0, "investments":0, "milestones":0,
                                     "products": 0,"acquisition": 0, "acquisitions": 0, "video_embeds": 0, 
                                     "screenshots": 0, "external_links": 0, "partners": 0, "image": 0
                                }
                                )

In [75]:
filter_companies = pd.DataFrame(query)

In [76]:
filter_companies.head(10)

,name,category_code,number_of_employees,founded_year,deadpooled_year,description,total_money_raised,offices
0,Mahalo,web,40,2007,None,Life long learning content,$21M,"[{'description': '', 'address1': '3525 Eastham..."
1,Livestream,games_video,120,2007,None,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ..."
2,Ustream,games_video,250,2007,None,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr..."
3,SodaHead,web,25,2007,None,Online Community and Discussion,$12.7M,"[{'description': '', 'address1': '16161 Ventur..."
4,GoingOn,software,40,2008,None,Private Academic Networks,$8.5M,"[{'description': 'GoingOn Networks, Inc.', 'ad..."
5,Dropbox,software,350,2007,None,"Always have your stuff, wherever you are",$257M,"[{'description': 'Headquarters', 'address1': '..."
6,Jimdo,web,170,2007,None,online website builder,$0,"[{'description': 'Germany', 'address1': 'Stres..."
7,Yola,web,70,2007,None,Free Websites,$25M,"[{'description': 'Yola San Francisco', 'addres..."
8,Docstoc,web,40,2007,None,The Online Resource for Small Businesses,$4M,"[{'description': '', 'address1': '409 S Santa ..."
9,Crunchyroll,games_video,50,2008,None,Digital Entertainment and Media Platform,$4.8M,"[{'description': '', 'address1': '', 'address2..."


In [77]:
filter_companies['offices']

0      [{'description': '', 'address1': '3525 Eastham...
1      [{'description': 'Livestream HQ', 'address1': ...
2      [{'description': 'San Francisco Office', 'addr...
3      [{'description': '', 'address1': '16161 Ventur...
4      [{'description': 'GoingOn Networks, Inc.', 'ad...
                             ...                        
322    [{'description': 'Tiverias Apps LLC.', 'addres...
323    [{'description': '', 'address1': '7th Floor,',...
324    [{'description': 'Headquarters', 'address1': '...
325    [{'description': '', 'address1': '143 Varick S...
326    [{'description': '', 'address1': '163 Hastings...
Name: offices, Length: 327, dtype: object

In [78]:
filter_companies = filter_companies[pd.Series(map(len, filter_companies["offices"])) > 0]

In [79]:
filter_companies["country"] = [e[0]["country_code"] for e in filter_companies["offices"]]
filter_companies["state"] = [e[0]["state_code"] for e in filter_companies["offices"]]
filter_companies["city"] = [e[0]["city"] for e in filter_companies["offices"]]

In [80]:
def getLocation(wendy):
    wendy = filter_companies['offices']
    longitude = wendy[0]['longitude']
    latitude = wendy[0]['latitude']
    loc = {
        'type':'Point',
        'coordinates':[float(longitude), float(latitude)]
    }
    return {
        "lat": wendy[0]['latitude'],
        "lng": wendy[0]['longitude'],
        "principal_office": loc
    }

In [81]:
latitude = [d[0].get('latitude') for d in filter_companies.offices]
longitude = [d[0].get('longitude') for d in filter_companies.offices]

In [82]:
filter_companies['latitude'] = latitude
filter_companies['longitude'] = longitude

In [83]:
filter_companies = filter_companies[~ filter_companies["latitude"].isna()]

In [84]:
filter_companies['latitude']

0      34.017606
1      40.726155
2      37.392936
3      37.269175
4      37.782263
         ...    
318    51.517356
320    39.612223
322    42.367287
325    40.726160
326    49.282455
Name: latitude, Length: 220, dtype: float64

In [85]:
city = [d[0].get('city') for d in filter_companies.offices]
country = [d[0].get('country_code') for d in filter_companies.offices]

In [86]:
filter_companies['city'] = city
filter_companies['country'] = country

In [87]:
filter_companies.head()

,name,category_code,number_of_employees,founded_year,deadpooled_year,description,total_money_raised,offices,country,state,city,latitude,longitude
0,Mahalo,web,40,2007,None,Life long learning content,$21M,"[{'description': '', 'address1': '3525 Eastham...",USA,CA,Culver City,34.017606,-118.487267
1,Livestream,games_video,120,2007,None,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",USA,NY,New York,40.726155,-73.995625
2,Ustream,games_video,250,2007,None,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",USA,CA,San Francisco,37.392936,-122.079480
3,SodaHead,web,25,2007,None,Online Community and Discussion,$12.7M,"[{'description': '', 'address1': '16161 Ventur...",USA,CA,Encino,37.269175,-119.306607
4,GoingOn,software,40,2008,None,Private Academic Networks,$8.5M,"[{'description': 'GoingOn Networks, Inc.', 'ad...",USA,CA,San Francisco,37.782263,-122.392142


In [88]:
filter_companies['total_money_raised']

0        $21M
1      $14.7M
2      $60.1M
3      $12.7M
4       $8.5M
        ...  
318    $2.56M
320        $0
322        $0
325        $0
326    $17.2M
Name: total_money_raised, Length: 220, dtype: object

In [89]:
def moneyFn(value):
    dCoin = {'CAD': 1.3124497992,'RUB': 63.7561301828, 'EUR': 0.8916629514, 'GBP': 0.7991529202}
    vMoney = {'K':1000, 'M':1000000, 'B': 100000000000}
    vNumber = float(re.search('[+-]?([0-9]*[.])?[0-9]+', value)[0])
    if value.endswith('B'):
        exchange = vNumber*(vMoney['B'])
    elif value.endswith('K[k]'):
        exchange = vNumber*(vMoney['K'])
    elif value.endswith('M'):
        exchange = vNumber*(vMoney['M'])
    elif value.startswith("C"):
        exchange =  vNumber*(dCoin['CAD'])
    elif value.startswith("$"):
        exchange =  vNumber
    elif value.startswith("€"):
        exchange = vNumber*(dCoin['EUR'])
    elif value.startswith("£"):
        exchange = vNumber*(dCoin['GBP'])
    elif value.startswith("r"):
        exchange = vNumber*(dCoin['RUB'])
    else:
        exchange = vNumber
    return int(exchange)

In [90]:
filter_companies['total_money_raised'] = filter_companies['total_money_raised'].apply(moneyFn)

In [91]:
filter_companies = filter_companies[filter_companies['country'] == 'USA']

In [92]:
filter_companies = filter_companies.reset_index(drop=True)

In [93]:
filter_companies.head()

,name,category_code,number_of_employees,founded_year,deadpooled_year,description,total_money_raised,offices,country,state,city,latitude,longitude
0,Mahalo,web,40,2007,None,Life long learning content,21000000,"[{'description': '', 'address1': '3525 Eastham...",USA,CA,Culver City,34.017606,-118.487267
1,Livestream,games_video,120,2007,None,Live streaming platform and website,14700000,"[{'description': 'Livestream HQ', 'address1': ...",USA,NY,New York,40.726155,-73.995625
2,Ustream,games_video,250,2007,None,Live Social Video Community,60100000,"[{'description': 'San Francisco Office', 'addr...",USA,CA,San Francisco,37.392936,-122.079480
3,SodaHead,web,25,2007,None,Online Community and Discussion,12700000,"[{'description': '', 'address1': '16161 Ventur...",USA,CA,Encino,37.269175,-119.306607
4,GoingOn,software,40,2008,None,Private Academic Networks,8500000,"[{'description': 'GoingOn Networks, Inc.', 'ad...",USA,CA,San Francisco,37.782263,-122.392142


In [94]:
filter_companies.to_json('./filtered_companies_clean.json', orient='records')